# Machine Learning memo

最近剛上完台大林軒田老師的Machine Learning Techniques和Machine Learning Foundations，總時程大概一個多月，因為時間有一點長，所以這裡把自己train過的data整理一下，因為不像real world data一樣要花大部分的時間做preprocessing，都是可以直接拿來train的data，所以這邊只有整理一些演算法(算演算法嗎？)。

### Perceptron Learning algorithm：

先看一下PLA的基本核心：

start from some $W_{0}$,

For t = 0,1.......

* find the next mistake of $W_{t}$ called ($X_{n(t)}$, $y_{n(t)}$):

$$sign(W_{t}^{T}X_{n(t)}) \neq y_{n(t)}$$

* correct the mistake by:

$$W_{t+1} \leftarrow W_{t} + y_{n(t)} X_{n(t)} $$

### 先load data

In [4]:
import numpy as np
import matplotlib.pyplot as plt

X_train15 = np.genfromtxt("/Users/huangyuchen/Desktop/ML_NTU/hw1_15_train.dat.txt")
X_train18 = np.genfromtxt("/Users/huangyuchen/Desktop/ML_NTU/hw1_18_train.dat.txt")
X_test18 = np.genfromtxt("/Users/huangyuchen/Desktop/ML_NTU/hw1_18_test.dat.txt")
## adding bias and initial W
X_train15 = np.insert(X_train15,0,1,1)
X_train18 = np.insert(X_train18,0,1,1)
X_test18 = np.insert(X_test18,0,1,1)

In [24]:
def PLA(X,y):
    error = 1
    m,n = X.shape
    W = np.zeros((1,n-1))
    while (error!=0):
        y = np.reshape(y,(m,1))
        for i in range(m):
            if np.sign(W.dot(X[i,:-1]))!= y[i]:
                W = W + y[i]*X[i,:-1]                
                error = min(error,np.mean(np.sign(X[:,:-1].dot(W.T))!=y))
                break
    
    return error

PLA(X_train15,X_train15[:,-1])

0.0

由結果可以看到，因為是設計好的data的關係，所以PLA會停下來，並且得到$E_{in}=0$，然而這並不是一個實用的演算法，因為很明顯現實中的data並不是這麼的單純，為了解決這樣子的問題，有另一個類似的algorithms:Pocket algorithms。

## Pocket Algorithms:

先看一下Pocket Algorithms的基本核心：

start from some $W_{0}$,

For t = 0,1.......

* find the (random) mistake of $W_{t}$ called ($X_{n(t)}$, $y_{n(t)}$):

$$sign(W_{t}^{T}X_{n(t)}) \neq y_{n(t)}$$

* modify W by:

$$W_{t+1} \leftarrow W_{t} + y_{n(t)} X_{n(t)} $$

如果$W_{t+1}$的結果比$W_{t}$好，就用$W_{t+1}$取代$W_{t}$，由於這次並沒有保證Pocket algorithms一定會停下來，所以停下來的決定是由我們相信足夠多次數的迭代決定的，這次就用X_train18來當做training samples。

In [23]:
def Pocket(X,y,X_test,y_test,itera):
    m,n = X.shape
    o,p =X_test.shape
    W = np.zeros((1,n-1))
    y = np.reshape(y,(m,1))
    y_test = np.reshape(y_test,(o,1))
    l = 0
    while l < itera:
        np.random.shuffle(X)
        error = np.mean(np.sign(X[:,:-1].dot(W.T))!=y)
        if np.sign(W.dot(X[0,:-1]))!= y[0]:
            W_ini = W + y[0]*X[0,:-1]                
            error_new = np.mean(np.sign(X[:,:-1].dot(W_ini.T))!=y)
            if error_new < error:
                W = W_ini
                continue
        l+=1
    return np.mean(np.sign(X_test[:,:-1].dot(W.T))!=y_test)

Pocket(X_train18,X_train18[:,-1],X_test18,X_test18[:,-1],5000)

0.252

因為有shuffle training data的關係，所以$E_{out}$並不會每次都一樣，因此我們試著看看平均的$E_{out}$是多少。

In [26]:
Eou = 0

for i in range(100):
    Eou+=Pocket(X_train18,X_train18[:,-1],X_test18,X_test18[:,-1],100) 
print("{:.3f}".format(Eou/100))

0.286
